In [1]:
import joblib
import logging
import warnings
import requests
import multiprocessing
import pandas as pd
import numpy as np
import yfinance as yf

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from zero_shot_learner import extend_df_with_cos_sim
from preprocessor import NewsPreprocessor
from preprocessor import transform_df
from contractions import contractions_dict
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger()
logger.disabled = True

I0721 14:49:13.561837 15980 file_utils.py:39] PyTorch version 1.2.0 available.
I0721 14:49:19.043715 15980 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0721 14:49:19.045713 15980 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0721 14:49:20.787230 15980 token

In [2]:
def load_stock(ticker_name, start_date="2012-01-01"):
    ticker = yf.Ticker(ticker_name)
    hist = ticker.history(period="max", start=start_date)
    hist.index = hist.index.set_names(['date'])
    hist = hist.reset_index(drop=False, inplace=False)
    hist["date"] = pd.to_datetime(hist["date"], utc=True)
    hist['date'] = hist['date'].apply(lambda x: x.date())
    hist.sort_values(by='date', inplace=True)
    hist.reset_index(drop=True, inplace=True)
    hist["label"] = hist["Close"].diff(periods=1)
    hist.dropna(inplace=True)
    hist["label"] = hist["label"].map(lambda x: 1 if float(x) >= 0 else 0)
    return hist

def load_news(df, labels, sort_by, k):
    """
    :param file_name: str
    :param labels: list of str (for zero-shot learner)
    :param sort_by: str (str in labels)
    :param k: int (top k news)
    :return: pandas dataframe
    """
    df.drop_duplicates(subset="title", inplace=True)
    preprocessor = NewsPreprocessor(contractions_dict=contractions_dict)
    df["clean_title"] = df["title"].apply(lambda x: preprocessor.ultimate_clean(x))
    df = extend_df_with_cos_sim(df=df, col="clean_title", labels=labels, sort_by=sort_by)
    df = transform_df(df=df, sort_by=sort_by, k=k)
    df.reset_index(drop=True, inplace=True)
    return df

In [3]:
def merge_function(df, ticker, return_dict):
    news = df[df["ticker"] == str(ticker)]
    news = load_news(news, labels=["stock"], sort_by="stock", k=5)
    stock = load_stock(str(ticker), start_date="2012-01-01")
    news_and_stock = pd.merge(news, stock, on=["date"])
    news_and_stock.set_index('date', inplace=True)
    return_dict[ticker] = news_and_stock

In [6]:
TRAIN_START_DATE = "2012-01-01"
TRAIN_END_DATE = "2015-12-31"
VALID_START_DATE = "2016-01-01"
VALID_END_DATE = "2016-12-31"
TEST_START_DATE = "2017-01-01"
TEST_END_DATE = "2020-07-01"

train = pd.DataFrame()
valid = pd.DataFrame()
test = pd.DataFrame()

df_merge = joblib.load("../data/sp500_top100_v1.bin")

for ticker in tqdm(df_merge["ticker"].unique()):
    news = df_merge[df_merge["ticker"] == str(ticker)]
    news = load_news(news, labels=["finance"], sort_by="finance", k=3)
    stock = load_stock(str(ticker), start_date="2012-01-01")
    news_and_stock = pd.merge(news, stock, on=["date"])
    news_and_stock.set_index('date', inplace=True)
    
    train_temp = news_and_stock.loc[pd.to_datetime(TRAIN_START_DATE).date():pd.to_datetime(TRAIN_END_DATE).date()]
    valid_temp = news_and_stock.loc[pd.to_datetime(VALID_START_DATE).date():pd.to_datetime(VALID_END_DATE).date()]
    test_temp = news_and_stock.loc[pd.to_datetime(TEST_START_DATE).date():pd.to_datetime(TEST_END_DATE).date()]
    
    train = pd.concat([train, train_temp], axis=0)
    valid = pd.concat([valid, valid_temp], axis=0)
    test = pd.concat([test, test_temp], axis=0)

I0721 14:49:49.498695 15980 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/deepset/sentence_bert/config.json from cache at C:\Users\YangWang/.cache\torch\transformers\1a89cc2d2dfc0e9beb7d49442e942849b45bbbf49d0b004f6be414b44c4e01fa.f9ed8a8332fc340fb779f9e83f1745369bbe51a3ac3e1c0d0dc5c3cf72ef4626
I0721 14:49:49.499730 15980 configuration_utils.py:321] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0721 14:49:49.500700 15980 tokenization_utils.py:929] Model name 'deepset/sentence_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base

Start zero-shot learner...
Done!
Start transforming dataframe...
Done!


I0721 14:55:38.667075 15980 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/deepset/sentence_bert/config.json from cache at C:\Users\YangWang/.cache\torch\transformers\1a89cc2d2dfc0e9beb7d49442e942849b45bbbf49d0b004f6be414b44c4e01fa.f9ed8a8332fc340fb779f9e83f1745369bbe51a3ac3e1c0d0dc5c3cf72ef4626
I0721 14:55:38.669073 15980 configuration_utils.py:321] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0721 14:55:38.670071 15980 tokenization_utils.py:929] Model name 'deepset/sentence_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base

Start zero-shot learner...
Done!
Start transforming dataframe...
Done!


I0721 14:57:47.968962 15980 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/deepset/sentence_bert/config.json from cache at C:\Users\YangWang/.cache\torch\transformers\1a89cc2d2dfc0e9beb7d49442e942849b45bbbf49d0b004f6be414b44c4e01fa.f9ed8a8332fc340fb779f9e83f1745369bbe51a3ac3e1c0d0dc5c3cf72ef4626
I0721 14:57:47.971965 15980 configuration_utils.py:321] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0721 14:57:47.972925 15980 tokenization_utils.py:929] Model name 'deepset/sentence_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base

...

Start zero-shot learner...
Done!
Start transforming dataframe...
Done!


I0721 16:54:57.770391 15980 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/deepset/sentence_bert/config.json from cache at C:\Users\YangWang/.cache\torch\transformers\1a89cc2d2dfc0e9beb7d49442e942849b45bbbf49d0b004f6be414b44c4e01fa.f9ed8a8332fc340fb779f9e83f1745369bbe51a3ac3e1c0d0dc5c3cf72ef4626
I0721 16:54:57.771424 15980 configuration_utils.py:321] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0721 16:54:57.771424 15980 tokenization_utils.py:929] Model name 'deepset/sentence_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base

Start zero-shot learner...
Done!
Start transforming dataframe...
Done!


I0721 16:55:07.697285 15980 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/deepset/sentence_bert/config.json from cache at C:\Users\YangWang/.cache\torch\transformers\1a89cc2d2dfc0e9beb7d49442e942849b45bbbf49d0b004f6be414b44c4e01fa.f9ed8a8332fc340fb779f9e83f1745369bbe51a3ac3e1c0d0dc5c3cf72ef4626
I0721 16:55:07.698144 15980 configuration_utils.py:321] Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

I0721 16:55:07.699137 15980 tokenization_utils.py:929] Model name 'deepset/sentence_bert' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base

Start zero-shot learner...
Done!
Start transforming dataframe...
Done!



In [ ]:
# manager = multiprocessing.Manager()
# return_dict = manager.dict()
# jobs = []
# for ticker in df_merge["ticker"].unique():
#     p = multiprocessing.Process(target=merge_function, args=(df_merge, ticker, return_dict))
#     jobs.append(p)
#     p.start()
# for proc in jobs:
#     proc.join()

In [7]:
joblib.dump(train, "../data/train.bin", compress=5)
joblib.dump(valid, "../data/valid.bin", compress=5)
joblib.dump(test, "../data/test.bin", compress=5)

['../data/test.bin']

In [8]:
train

,Top 1 News,Top 2 News,Top 3 News,Open,High,Low,Close,Volume,Dividends,Stock Splits,label
date,,,,,,,,,,,
2012-01-04,apple hires adobe officer to lead iad: report,NaN,NaN,50.71,51.28,50.62,51.13,65005500.0,0.0,0.0,1
2012-01-05,refile-taiwan is elan says apple to pay $5 mln...,taiwan is elan says apple to pay $5 million in...,update 1-apple to pay elan $5 mln to settle pa...,51.32,51.76,51.04,51.70,67817400.0,0.0,0.0,1
2012-01-09,apple is cook got rich stock award worth $376 ...,apple is siri puts voice-enabled search in spo...,chinese authors sue apple for copyright infrin...,52.62,52.90,52.11,52.16,98506100.0,0.0,0.0,0
2012-01-10,"kodak sues apple, htc over digital image patents",NaN,NaN,52.67,52.68,52.13,52.34,64549100.0,0.0,0.0,1
2012-01-11,strong apple contingent expected at ces,corrected-ces-strong apple contingent expected...,apple plans january 19 education event in new ...,52.27,52.29,51.86,52.26,53771200.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
2014-02-24,fitch upgrades fidelity national information s...,NaN,NaN,49.69,50.60,49.69,50.21,1004200.0,0.0,0.0,1
2014-01-29,servicenow is revenue jumps 67 pct,servicenow is revenue jumps 67 percent,NaN,58.19,58.22,56.44,57.60,1998900.0,0.0,0.0,0
2014-04-24,corrected-servicenow 1st-quarter revenue beats...,"servicenow sees second-quarter, 2014 revenue a...","update 1-servicenow sees q2, 2014 revenue abov...",54.70,54.70,45.07,49.88,10440700.0,0.0,0.0,0


In [9]:
valid

,Top 1 News,Top 2 News,Top 3 News,Open,High,Low,Close,Volume,Dividends,Stock Splits,label
date,,,,,,,,,,,
2016-01-04,apple shares off but weather u.s. market selloff,NaN,NaN,95.40,97.97,94.83,97.95,67649400.0,0.0,0.0,1
2016-01-05,"apple expected to cut iphone 6s, 6s plus produ...","update 2-apple expected to cut iphone 6s, 6s p...","apple expected to cut iphone 6s, 6s plus produ...",98.32,98.41,95.22,95.49,55791000.0,0.0,0.0,0
2016-01-06,update 1-apple paid ceo tim cook $10.3 mln in ...,apple paid ceo tim cook $10.3 mln in 2015,apple paid ceo tim cook $10.3 million in 2015,93.49,95.18,92.85,93.63,68457400.0,0.0,0.0,0
2016-01-07,apple buys artificial intelligence startup emo...,update 1-apple suppliers cut revenue estimates...,NaN,91.75,93.10,89.66,89.67,81094400.0,0.0,0.0,0
2016-01-08,"apple registers automobile domain names, inclu...",update 1-major apple supplier hon hai of taiwa...,major apple supplier hon hai of taiwan posts 2...,91.63,92.15,89.96,90.15,70798000.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2016-04-21,brief-servicenow q1 non-gaap earnings per shar...,NaN,NaN,74.77,76.84,73.21,74.27,8395500.0,0.0,0.0,1
2016-06-01,brief-servicenow acquires brightpoint security,NaN,NaN,71.38,73.50,71.06,73.17,1889700.0,0.0,0.0,1
2016-07-28,brief-servicenow q2 earnings per share view $0...,NaN,NaN,75.40,78.77,74.15,76.38,3991900.0,0.0,0.0,1


In [10]:
test

,Top 1 News,Top 2 News,Top 3 News,Open,High,Low,Close,Volume,Dividends,Stock Splits,label
date,,,,,,,,,,,
2017-01-04,apple confirms $1 bln investment in softbank t...,update 2-apple confirms $1 bln investment in s...,apple confirms $1 billion investment in softba...,110.11,110.73,110.01,110.27,21118100.0,0.0,0.0,0
2017-01-05,brief-apple says app store generated over $20 ...,update 1-apple is app store generated $20 bln ...,apple is app store generated $20 billion for d...,110.17,111.07,110.07,110.83,22193600.0,0.0,0.0,1
2017-01-06,brief-apple inc is ceo tim cook is total 2016 ...,canada is competition watchdog closes two-year...,NaN,110.99,112.30,110.70,112.07,31751900.0,0.0,0.0,1
2017-01-10,china is wechat seeks slice of apple is app st...,china is wechat seeks slice of apple is app st...,tesla taps apple engineer for autopilot software,112.88,113.46,112.44,113.21,24462100.0,0.0,0.0,1
2017-01-12,u.s. appeals court revives antitrust lawsuit a...,NaN,NaN,113.01,113.39,112.35,113.34,27086200.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-03-17,brief-servicenow releases four emergency respo...,NaN,NaN,271.41,302.98,270.02,287.42,3908100.0,0.0,0.0,1
2020-04-21,brief-lincoln financial announces steps to sup...,NaN,NaN,298.96,303.70,281.57,288.77,2892800.0,0.0,0.0,0
2020-04-30,"u.s. research roundup- labcorp, servicenow, ze...",NaN,NaN,327.84,357.78,327.49,351.54,5387700.0,0.0,0.0,1
